1、实现K近邻算法

2、采用K近邻对1个数据集进行分类分析，进行分类训练和测试，并给出分析结果

注意：作业提交需包含所有代码，必要时可提交压缩包，代码尽可能在(姓名_KNN.ipynb)中

数据集可以采用阿尔及利亚森林火灾数据集：UCI Machine Learning Repository: Algerian Forest Fires Dataset Data Set

In [103]:
import pandas as pd 
import numpy as np

In [104]:
from this import d
df=pd.read_csv('./dataset/Algerian Forest Fires Dataset Data Set/Algerian_forest_fires_dataset_UPDATE.csv',header=1)
df.head()
df=df.drop([122,123])
df.iloc[121:125]

,day,month,year,Temperature,RH,Ws,Rain,FFMC,DMC,DC,ISI,BUI,FWI,Classes
121,30,09,2012,25,78,14,1.4,45,1.9,7.5,0.2,2.4,0.1,not fire
124,01,06,2012,32,71,12,0.7,57.1,2.5,8.2,0.6,2.8,0.2,not fire
125,02,06,2012,30,73,13,4,55.7,2.7,7.8,0.6,2.9,0.2,not fire
126,03,06,2012,29,80,14,2,48.7,2.2,7.6,0.3,2.6,0.1,not fire


In [105]:
df.reset_index(drop=True)
df.iloc[121:125]

,day,month,year,Temperature,RH,Ws,Rain,FFMC,DMC,DC,ISI,BUI,FWI,Classes
121,30,09,2012,25,78,14,1.4,45,1.9,7.5,0.2,2.4,0.1,not fire
124,01,06,2012,32,71,12,0.7,57.1,2.5,8.2,0.6,2.8,0.2,not fire
125,02,06,2012,30,73,13,4,55.7,2.7,7.8,0.6,2.9,0.2,not fire
126,03,06,2012,29,80,14,2,48.7,2.2,7.6,0.3,2.6,0.1,not fire


In [ ]:
dsasd